**Пайплайн:**

- ✅ Сначала нам надо скачать дату – соберите как минимум 60 (30 положительных и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления " тонального словаря" (чем больше отзывов, тем лучше) и 10 отзывов для проверки качества.


- ✅ Токенизируйте слова, приведите их к нижнему регистру и к начальной форме (1 балл за токенизацию, 1 - за начальную форму)


- ✅ Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (3 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)


- ✅ Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1 - за коректно работающую функцию, 1 - за подсчёт accuracy)


- Предложите как минимум 2 способа улучшить эту программу с помощью добавления к ней любых мулек - просто словами, писать улучшающий код не надо (1 балл)

# №1 Parsing data

Cобираем >60 (30 положительных и 30 отрицательных) отзывов на похожие продукты  для составления "тонального словаря" и 10 отзывов для проверки качества.

**Берём отзывы на рязанские отели с сайта tripadvisor.ru**

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from sklearn.metrics import accuracy_score

from collections import Counter
from pymystem3 import Mystem
import pandas as pd
import nltk
from tqdm import tqdm
import time
import csv
import os
import re

m = Mystem()

In [2]:
firefoxdriver_path = '/Users/mariabocharova/PycharmProjects/REALEC/firefoxdriver'
chromedriver_path = '/Users/mariabocharova/PycharmProjects/REALEC/chromedriver'

hotels = ['https://www.tripadvisor.ru/Hotel_Review-g298485-d1732311-Reviews-Congress_Hotel_Forum-Ryazan_Ryazan_Oblast_Central_Russia.html', 
          'https://www.tripadvisor.ru/Hotel_Review-g15512259-d6533471-Reviews-Tourist_Base_Polushkino-Polushkino_Ryazan_Oblast_Central_Russia.html', 
          'https://www.tripadvisor.ru/Hotel_Review-g298485-d7257464-Reviews-Hostel_Like-Ryazan_Ryazan_Oblast_Central_Russia.html', 
          'https://www.tripadvisor.ru/Hotel_Review-g298485-d10817084-Reviews-Kremlyovskiy-Ryazan_Ryazan_Oblast_Central_Russia.html', 
          'https://www.tripadvisor.ru/Hotel_Review-g298485-d1469261-Reviews-Aragon_Hotel-Ryazan_Ryazan_Oblast_Central_Russia.html',
          'https://www.tripadvisor.ru/Hotel_Review-g298485-d5534355-Reviews-V_Nekotorom_Tsarstve_Hotel-Ryazan_Ryazan_Oblast_Central_Russia.html',
          'https://www.tripadvisor.ru/Hotel_Review-g298485-d7133243-Reviews-Hotel_Ryazan-Ryazan_Ryazan_Oblast_Central_Russia.html']

In [3]:
df_reviews = pd.DataFrame(columns=['positive', 'negative'])

In [4]:
def get_reviews(hotel_link, save_to_path, firefoxdriver_path='/Users/mariabocharova/PycharmProjects/REALEC/firefoxdriver'):
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')
    driver = webdriver.Firefox(executable_path=firefoxdriver_path)
    driver.get(hotel_link)
    time.sleep(5)

    with open(save_to_path, 'a') as csvfile:
        writer = csv.writer(csvfile)
        try:
            while driver.find_element_by_xpath(".//a[@class='ui_button nav next primary ']"):
                # Нахожу блок с оценкой и отзывом
                reviews = driver.find_elements_by_xpath("//div[@data-test-target='HR_CC_CARD']")
                for review in reviews:
                    text = review.find_element_by_xpath(".//q[@class='XllAv H4 _a']").text
                    score = float(review.find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]")
                                .get_attribute("class").strip('ui_bubble_rating bubble_'))
                    # Сохраняю оценку и комментарий
                    writer.writerow([score / 10, text])
                driver.find_element_by_xpath(".//a[@class='ui_button nav next primary ']").click()
                time.sleep(7)
        except Exception as e:
            pass
    driver.close()

In [ ]:
with open('tripadvisor_reviews.csv', 'a') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['score', 'text'])
for link in tqdm(hotels):
    get_reviews(link, 'tripadvisor_reviews.csv')

**Всего отзывов:**

In [5]:
df = pd.read_csv('tripadvisor_reviews.csv')
len(df)

615

Будем считать, что отзыв является положительным, если его оценивают на 4-5. Если оценка меньше – то отзыв отрицательный. 

In [6]:
df['score'].value_counts(sort=True)

5.0    309
4.0    183
3.0     72
1.0     29
2.0     22
Name: score, dtype: int64

Выберем рандомные 120 положительных и 120 отрицательных отзывов. 

In [7]:
df_positive = df.loc[df['score'] >= 4].sample(n = 120)
df_negative = df.loc[df['score'] < 4].sample(n = 120)

**Достанем отдельные отзывы для предсказания**

In [ ]:
link = 'https://www.tripadvisor.ru/Hotel_Review-g298485-d6851547-Reviews-Esenin-Ryazan_Ryazan_Oblast_Central_Russia.html'
get_reviews(link, 'tripadvisor_reviews_prediction.csv')

In [8]:
def lemmatize(text):
    lemm_text = ''
    for word in m.lemmatize(text):
        if word.isalpha():
            lemm_text += f'{word} '
    return lemm_text

In [9]:
with open('tripadvisor_reviews_prediction.csv', encoding='utf-8') as f:
    next(f)
    negative_predict = [lemmatize(line[5:-2]).strip() for line in f if float(line[0:3]) < 4][:10]

with open('tripadvisor_reviews_prediction.csv', encoding='utf-8') as f:
    next(f)
    positive_predict = [lemmatize(line[5:-2]).strip() for line in f if float(line[0:3]) >= 4][:10]      

In [10]:
df_predict = pd.read_csv('tripadvisor_reviews_prediction.csv')
df_predict['score'].value_counts(sort = True)

4.0    34
5.0    32
3.0    28
2.0    11
1.0    10
Name: score, dtype: int64

# №2 Токенизация

Токенизируем слова, приводим их к нижнему регистру и к начальной форме.

In [11]:
df_positive['text_lemm'] = df_positive['text'].apply(lemmatize)
df_negative['text_lemm'] = df_negative['text'].apply(lemmatize)

In [12]:
df_positive.head(3)

,score,text,text_lemm
516,4.0,"Отель подкупает отличным соотношением ""цена/ка...",отель подкупать отличный соотношение цена каче...
4,5.0,Отель замечательный. В номере есть холодильник...,отель замечательный в номер быть холодильник ч...
511,5.0,Побывали в этом отеле.Стандартный номер.Все но...,побывать в этот отель стандартный номер весь н...


# №3 Множества

Составляем 2 множества – в одном  слова, которые встречаются только в положительных отзывах, а в другом – встречающиеся только в отрицательных. 

In [13]:
def create_sets_of_words(df_positive, df_negative):
    positive_reviews = Counter(' '.join(df_positive['text_lemm']).split())
    negative_reviews = Counter(' '.join(df_negative['text_lemm']).split())

    positive_reviews = set({word: n for word, n in positive_reviews.items() if n > 1})
    negative_reviews = set({word: n for word, n in negative_reviews.items() if n > 1})

    positive_words_set = positive_reviews - negative_reviews
    negative_words_set = negative_reviews - positive_reviews
    
    return positive_words_set, negative_words_set

# №4 Предсказание

Создаем функцию, которая будет определять, положительный отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и считаем качество при помощи accuracy (1 - за коректно работающую функцию, 1 - за подсчёт accuracy)

In [14]:
def predict_tone(text, positive_words_set, negative_words_set):
    text = lemmatize(text).split()
    positive_num = len([word for word in text if word in positive_words_set])
    negative_num = len([word for word in text if word in negative_words_set])
    positive_percent = round(positive_num / len(text) * 100, 2)
    negative_percent = round(negative_num / len(text) * 100, 2)
    
    return positive_percent, negative_percent

**Считаем accuracy для функции определения негативных отзывов**

In [18]:
neg_gold = [1] * 10
neg_results = []

for prediction in negative_predict:
    positive_words_set, negative_words_set = create_sets_of_words(df_positive, df_negative)
    pos, neg = predict_tone(prediction, positive_words_set, negative_words_set)
    if neg > pos:
        neg_results.append(1)
    else:
        neg_results.append(0)
print('Accuracy of detecting negative reviews:', accuracy_score(neg_gold, neg_results))

Accuracy of detecting negative reviews: 0.9


**Считаем accuracy для функции определения положительных отзывов**

In [16]:
pos_gold = [1] * 10
pos_results = []

for prediction in positive_predict:
    positive_words_set, negative_words_set = create_sets_of_words(df_positive, df_negative)
    pos, neg = predict_tone(prediction, positive_words_set, negative_words_set)
    if pos > neg:
        pos_results.append(1)
    else:
        pos_results.append(0)
print('Accuracy of detecting positive reviews:', accuracy_score(pos_gold, pos_results))

Accuracy of detecting positive reviews: 0.3


# Limitations, или как попробовать улучшить 

1. Можно попробовать использовать сеты из биграмм, а не из единичных терминов. Тогда, например, сеты составленные из отзывов _мне не понравился отель_ и _мне понравился отель_ будут отличаться не только наличием/отсутствием частицы _не_, по которой нельзя понять, к чему она относится. 
2. Можно добавить POS tags и учитывать при анализе только Noun, Adjective, Adverb, Verb.
